## MGMTMSA 403 Homework 2: Portfolio Optimization


## Pre-processing step: Estimate expected returns and covariance

In [1]:
# Import gurobi and numpy
from gurobipy import *
import numpy as np
from numpy import genfromtxt
import csv
import pandas as pd

## Get index of 4 tickers
tick4 = ["MSFT","GS","PG","SCHP"];

# Get variable names
with open('Prices.csv') as csvFile:
    reader = csv.reader(csvFile)
    tickers = next(reader) ## stores the tickers of all 390 stocks

tickind =[];
for t in tick4:
    tickind.append(tickers.index(t)) ## retrieve index that corresponds to each ticker

# Load data
prices = genfromtxt('Prices.csv', delimiter=',',skip_header = 1)

# get dimensions of data
d = prices.shape[0]
n = prices.shape[1]

# calculate monthly returns of each stock
returns = np.zeros((d-1,n))
for stock in range(n):
    for month in range(d-1):
        returns[month,stock] = prices[month+1,stock]/prices[month,stock]-1
        
# Store average return (parameter r_i in portfolio optimization model)       
avg_return = np.zeros(n)
avg_return = np.mean(returns,axis=0)

# Store covariance matrix (parameter C_ij in portfolio optimization model)
C = np.zeros((n,n))
C = np.cov(np.transpose(returns))


In [2]:
# making data frame and extracting stock tickers 
data = pd.read_csv("Prices.csv") 
stock_ticker = data.columns
stock_ticker

Index(['MMM', 'ABT', 'ABBV', 'ABMD', 'ACN', 'ATVI', 'ADBE', 'AMD', 'AAP',
       'AES',
       ...
       'PWR', 'QCOM', 'DGX', 'RL', 'RJF', 'RTN', 'O', 'REG', 'SCHP', 'BND'],
      dtype='object', length=390)

## Question 1. Formulate and solve each of the three models in Python, and then answer the following questions. For each part, also include your Gurobi output.

#### **Model 1**  
Start by focusing on a four-asset portfolio: Suppose you can only invest in Microsoft (MSFT), Goldman Sachs (GS), Proctor & Gamble (PG), and U.S. Treasury Bonds (SCHP). Con- struct a minimum-variance portfolio with an expected monthly return of at least 0.5%. Write down the optimal risk (i.e. the optimal objective function value), solver time, and the weight on each of the four stocks.**

In [3]:
#Initialize Model
mod = Model() 

#Tick4 Average Return 
tick4_avg_return = avg_return[tickind]

#Tick4 Covaraince Matrix 
tick4_covariance_matrix = C[tickind][:,tickind]

#Define Data 
stocks = 4
mim_return  = 0.005

#Define decision variables
weight = mod.addVars(stocks)

#Define Constraints
return_contraint = mod.addConstr(sum(weight[i]*tick4_avg_return[i] for i in range(stocks)) >= mim_return)

Weights_add_up_to_1_contraint = mod.addConstr(sum(weight[i] for i in range(stocks)) == 1)

for i in range(stocks):
    Non_negativity_contraint = mod.addConstr(weight[i] >= 0)
    
#Create the objective function, and set it to be minimized.
mod.setObjective(sum(weight[i] * weight[j] * tick4_covariance_matrix[i,j] for i in range(stocks) for j in range(stocks)), GRB.MINIMIZE)
mod.update()
mod.optimize()

Set parameter Username
Academic license - for non-commercial use only - expires 2022-03-09
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 6 rows, 4 columns and 12 nonzeros
Model fingerprint: 0x70c79b8a
Model has 10 quadratic objective terms
Coefficient statistics:
  Matrix range     [2e-04, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [5e-05, 7e-03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-03, 1e+00]
Presolve removed 4 rows and 0 columns
Presolve time: 0.02s
Presolved: 2 rows, 4 columns, 8 nonzeros
Presolved model has 10 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 3
 AA' NZ     : 1.000e+01
 Factor NZ  : 1.500e+01
 Factor Ops : 5.500e+01 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Ti

In [4]:
# Retrieve optimal value and optimal solution
opt_val = mod.objval
print(f"The Optimal Minimum Risk is {opt_val:.5%}\n")

# Solver Time 
print(f"The Solver Time is {mod.Runtime} seconds\n")

x_opt = pd.DataFrame([weight[i].x for i in range(stocks)])
x_opt.columns =['Profolio Weight in %']
x_opt.index = tick4
x_opt.round(5)*100

The Optimal Minimum Risk is 0.01775%

The Solver Time is 0.05010199546813965 seconds



,Profolio Weight in %
MSFT,23.712
GS,2.586
PG,0.000
SCHP,73.702


#### **Model 2**  
Now suppose you can invest in all 390 stocks. Construct a minimum-variance portfolio with an expected monthly return of at least 0.5%.Write down the optimal risk and solver time**

In [5]:
# Initialize Model
mod2 = Model() 

# Define Data 
stocks = 390
mim_return  = 0.005

# Define decision variables
weight2 = mod2.addVars(stocks)

# Define Constraints
return_contraint = mod2.addConstr(sum(weight2[i]*avg_return[i] for i in range(stocks)) >= mim_return)

Weights_add_up_to_1_contraint = mod2.addConstr(sum(weight2[i] for i in range(stocks)) == 1)

for i in range(stocks):
    Non_negativity_contraint = mod2.addConstr(weight2[i] >= 0)
    
# Create the objective function, and set it to be minimized.
mod2.setObjective(sum(weight2[i] * weight2[j] * C[i,j] for i in range(stocks) for j in range(stocks)), GRB.MINIMIZE)
mod2.update()
mod2.optimize()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 392 rows, 390 columns and 1170 nonzeros
Model fingerprint: 0xd3a53ccf
Model has 76245 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e-06, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-07, 8e-02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-03, 1e+00]
Presolve removed 390 rows and 0 columns
Presolve time: 0.02s
Presolved: 2 rows, 390 columns, 780 nonzeros
Presolved model has 76245 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 59
 AA' NZ     : 1.830e+03
 Factor NZ  : 1.891e+03
 Factor Ops : 7.753e+04 (less than 1 second per iteration)
 Threads    : 2

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   2.89821559e-13 -2.89821559e-13  3.90e+05 3.22e-07  1.00e+06   

In [6]:
# Retrieve optimal value and optimal solution
opt_val2 = mod2.objval
print(f"The Optimal Minimum Risk is {opt_val2:.5%}\n")

# Solver Time 
print(f"The Solver Time is {mod2.Runtime} seconds\n")

# Profolio Weight in Percentage by Stock
x_opt2 = pd.DataFrame([weight2[i].x for i in range(stocks)])
x_opt2.columns =['Profolio Weight in %']
x_opt2.index = stock_ticker
x_opt2.round(5)*100

The Optimal Minimum Risk is 0.00288%

The Solver Time is 0.08627510070800781 seconds



,Profolio Weight in %
MMM,0.000
ABT,0.000
ABBV,1.136
ABMD,0.666
ACN,0.000
...,...
RTN,0.000
O,0.000
REG,0.000
SCHP,0.000


#### **Model 3** 
In practice, there are transaction fees associated with buying stocks. One way of keeping transaction fees low while still attaining desirable performance is to limit the total number of stocks that are purchased (i.e. limit the number of stocks that have a strictly positive weight). Construct a minimum-variance portfolio that selects at most 4 of the 390 stocks, and has an expected monthly return of at least 0.5%. (Note: By introducing binary variables into a quadratic program, we obtain a quadratic integer program. Fortunately, this particular quadratic integer program can be solved by Gurobi.)Report the optimal risk, solver time, and the ticker and weight on each of the four stocks selected by the model.

In [7]:
# Initialize Model
mod3 = Model() 

# Define Data 
stocks = 390
mim_return  = 0.005

# Define decision variables
weight3 = mod3.addVars(stocks)
selection3 = mod3.addVars(stocks, vtype = GRB.BINARY)


# Define Constraints
return_constraint = mod3.addConstr(sum(weight3[i]*avg_return[i] for i in range(stocks)) >= mim_return)

Weights_add_up_to_1_constraint = mod3.addConstr(sum(weight3[i] for i in range(stocks)) == 1)

for i in range(stocks):
    Non_negativity_constraint = mod3.addConstr(weight3[i] >= 0)

selection_at_most_4_constrait = mod3.addConstr(sum(selection3[i] for i in range(stocks)) <= 4) 

for i in range(stocks):
    select_when_weight_greater_than_zero_constriant = mod3.addConstr(selection3[i] >= weight3[i]) 
    
# Create the objective function, and set it to be minimized.
mod3.setObjective(sum(weight3[i] * weight3[j] * C[i,j] for i in range(stocks) for j in range(stocks)), GRB.MINIMIZE)
mod3.update()
mod3.optimize()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 783 rows, 780 columns and 2340 nonzeros
Model fingerprint: 0xe4bbc2cf
Model has 76245 quadratic objective terms
Variable types: 390 continuous, 390 integer (390 binary)
Coefficient statistics:
  Matrix range     [1e-06, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-07, 8e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-03, 4e+00]
Found heuristic solution: objective 0.0136011
Presolve removed 390 rows and 0 columns
Presolve time: 0.02s
Presolved: 393 rows, 780 columns, 1950 nonzeros
Presolved model has 76245 quadratic objective terms
Variable types: 390 continuous, 390 integer (390 binary)

Root relaxation: objective 2.878501e-05, 129 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    Bes

In [8]:
# Retrieve optimal value and optimal solution
opt_val3 = mod3.objval
print(f"The Optimal Minimum Risk is {opt_val3:.5%}\n")

# Solver Time 
print(f"The Solver Time is {mod3.Runtime} seconds\n")

# Profolio Weight in Percentage by Stock
x_opt3 = pd.DataFrame([weight3[i].x for i in range(stocks)])
x_opt3.columns =['Profolio Weight in %']
x_opt3.index = stock_ticker
df = x_opt3.round(5)*100 
df3 = df['Profolio Weight in %'] > 0 
df[df3]

The Optimal Minimum Risk is 0.00675%

The Solver Time is 76.39827108383179 seconds



,Profolio Weight in %
CME,12.641
LLY,7.548
NVDA,4.375
BND,75.436


## 2. Use your solution to Question 1 above to answer the following questions:

a) Is the optimal risk in Model 1 higher or lower than Model 2? Explain why in 1-2 sentences.

The optimal risk in Model 1 is higher than Model 2 beacuse Model 1 can only select 4 specified stocks and it ,thus,lacks portfolio diversification.

b) Is the optimal risk in Model 2 higher or lower than Model 3? Explain why in 1-2 sentences

The optimal risk in Model 2 is lower than Model 3 becuase Model 3 can only select up to 4 stocks given the question instruction, and that limits the profolio capability to diversify its components. 

## 3. In some cases, we may want to get an approximate solution quickly by terminating the branch- and-bound algorithm before it finds an optimal solution. There are two ways to terminate Gurobi early: (a) by setting a maximum time limit, and (b) by setting a maximum acceptable optimality gap (the tolerance). Use Model 3 to answer the following two questions. For each part, also include your Gurobi output.


a) Set Gurobi to terminate after 30 seconds by including XYZ.Params.TimeLimit = 30.0 in your code for Model 3, where ’XYZ’ is the name of your model. How does the objective function value at termination compare the optimal value obtained in question 1c)?

In [9]:
# Initialize Model
mod3 = Model() 

# Define Data 
stocks = 390
mim_return  = 0.005

# Define decision variables
weight3 = mod3.addVars(stocks)
selection3 = mod3.addVars(stocks, vtype = GRB.BINARY)


# Define Constraints
return_constraint = mod3.addConstr(sum(weight3[i]*avg_return[i] for i in range(stocks)) >= mim_return)

Weights_add_up_to_1_constraint = mod3.addConstr(sum(weight3[i] for i in range(stocks)) == 1)

for i in range(stocks):
    Non_negativity_constraint = mod3.addConstr(weight3[i] >= 0)

selection_at_most_4_constrait = mod3.addConstr(sum(selection3[i] for i in range(stocks)) <= 4) 

for i in range(stocks):
    select_when_weight_greater_than_zero_constriant = mod3.addConstr(selection3[i] >= weight3[i]) 

# Set Gurobi to terminate after 30 seconds    
mod3.Params.TimeLimit = 30.0
    
# Create the objective function, and set it to be minimized.
mod3.setObjective(sum(weight3[i] * weight3[j] * C[i,j] for i in range(stocks) for j in range(stocks)), GRB.MINIMIZE)
mod3.update()
mod3.optimize()

Set parameter TimeLimit to value 30
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 783 rows, 780 columns and 2340 nonzeros
Model fingerprint: 0xe4bbc2cf
Model has 76245 quadratic objective terms
Variable types: 390 continuous, 390 integer (390 binary)
Coefficient statistics:
  Matrix range     [1e-06, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-07, 8e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-03, 4e+00]
Found heuristic solution: objective 0.0136011
Presolve removed 390 rows and 0 columns
Presolve time: 0.02s
Presolved: 393 rows, 780 columns, 1950 nonzeros
Presolved model has 76245 quadratic objective terms
Variable types: 390 continuous, 390 integer (390 binary)

Root relaxation: objective 2.878501e-05, 129 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  

In [10]:
# Retrieve optimal value and optimal solution
opt_val3 = mod3.objval
print(f"The Optimal Minimum Risk is {opt_val3:.5%}\n")

The Optimal Minimum Risk is 0.00675%



The objective function value at termination is the same as the optimal value obtained in question 1c)

b) Set Gurobi to terminate after reaching a gap of 10% by including XYZ.Params.MIPGap = 0.1 in your code for Model 3, where ’XYZ’ is the name of your model. (Note: The default gap in Gurobi is 0.0001.) How does the solver time compare with the solution time obtained in question 1c)?

In [11]:
# Initialize Model
mod3 = Model() 

# Define Data 
stocks = 390
mim_return  = 0.005

# Define decision variables
weight3 = mod3.addVars(stocks)
selection3 = mod3.addVars(stocks, vtype = GRB.BINARY)


# Define Constraints
return_constraint = mod3.addConstr(sum(weight3[i]*avg_return[i] for i in range(stocks)) >= mim_return)

Weights_add_up_to_1_constraint = mod3.addConstr(sum(weight3[i] for i in range(stocks)) == 1)

for i in range(stocks):
    Non_negativity_constraint = mod3.addConstr(weight3[i] >= 0)

selection_at_most_4_constrait = mod3.addConstr(sum(selection3[i] for i in range(stocks)) <= 4) 

for i in range(stocks):
    select_when_weight_greater_than_zero_constriant = mod3.addConstr(selection3[i] >= weight3[i]) 

# Set Gurobi to terminate after reaching a gap of 10%s    
mod3.Params.MIPGap = 0.1
    
# Create the objective function, and set it to be minimized.
mod3.setObjective(sum(weight3[i] * weight3[j] * C[i,j] for i in range(stocks) for j in range(stocks)), GRB.MINIMIZE)
mod3.update()
mod3.optimize()

Set parameter MIPGap to value 0.1
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 783 rows, 780 columns and 2340 nonzeros
Model fingerprint: 0xe4bbc2cf
Model has 76245 quadratic objective terms
Variable types: 390 continuous, 390 integer (390 binary)
Coefficient statistics:
  Matrix range     [1e-06, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-07, 8e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-03, 4e+00]
Found heuristic solution: objective 0.0136011
Presolve removed 390 rows and 0 columns
Presolve time: 0.02s
Presolved: 393 rows, 780 columns, 1950 nonzeros
Presolved model has 76245 quadratic objective terms
Variable types: 390 continuous, 390 integer (390 binary)

Root relaxation: objective 2.878501e-05, 129 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Ob

In [12]:
# Solver Time 
print(f"The Solver Time is {mod3.Runtime} seconds\n")

The Solver Time is 64.6978108882904 seconds



The solver time is lower than the solution time obtained in question 1c